In [1]:
# skorch : pytorch를 쉽게 구현할 수 있는 라이브러리
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [2]:
X_train= X_train.reshape(-1,784)
X_test= X_test.reshape(-1,784)
X_train=X_train/255.
X_test=X_test/255.

In [3]:
import torch
X_train=torch.from_numpy(X_train).float()
y_train=torch.from_numpy(y_train.astype('int32')).long()
X_test=torch.from_numpy(X_test).float()
y_test=torch.from_numpy(y_test.astype('int32')).long()
print(X_train.shape)
print(X_test.shape)
X_train=X_train.cuda()
y_train=y_train.cuda()
X_test=X_test.cuda()
y_test=y_test.cuda()

torch.Size([60000, 784])
torch.Size([10000, 784])


In [4]:
import torch.nn as nn
import torch.nn.functional as F
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)
model=Net().cuda()

In [5]:
# pip install skorch
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(Net,max_epochs=20,lr=0.1)
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6427       0.8982        0.3508  1.8528
      2        0.3172       0.9162        0.2894  1.7737
      3        0.2669       0.9264        0.2514  1.9110
      4        0.2309       0.9355        0.2225  2.3442
      5        0.2033       0.9412        0.2004  5.2309
      6        0.1812       0.9477        0.1820  2.5707
      7        0.1630       0.9516        0.1671  4.1100
      8        0.1478       0.9553        0.1552  4.6161
      9        0.1349       0.9585        0.1453  1.7451
     10        0.1239       0.9609        0.1372  1.7229
     11        0.1144       0.9622        0.1307  1.7932
     12        0.1062       0.9637        0.1250  1.8123
     13        0.0990       0.9651        0.1201  1.9588
     14        0.0926       0.9652        0.1159  1.7137
     15        0.0870       0.9661        0.1122  1.6881
     16        0.0818       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=784, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  ),
)

In [6]:
import numpy as np
# 모형의 정확도 계산
pred = net.predict(X_test)
y_test = y_test.cpu().numpy()
accuracy = np.mean(pred == y_test)
accuracy

0.9731

In [7]:
from torchinfo import summary
summary(model, input_size=(100,784))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [100, 10]                 --
├─Linear: 1-1                            [100, 128]                100,480
├─Linear: 1-2                            [100, 10]                 1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
Total mult-adds (M): 10.18
Input size (MB): 0.31
Forward/backward pass size (MB): 0.11
Params size (MB): 0.41
Estimated Total Size (MB): 0.83